# Global AI Hub Project


Veri setini açıkla, her kolonu. 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Feature Engineering

### 1. Application Record Dataset Features

burda bu verisetini açıkla biraz


In [2]:
application= pd.read_csv(r"C:\Users\selve\Desktop\application_record.csv")
application.head(5)

,ID,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0
3,5008808,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0
4,5008809,F,N,Y,0,270000.0,Commercial associate,Secondary / secondary special,Single / not married,House / apartment,-19110,-3051,1,0,1,1,Sales staff,1.0


In [3]:
application.shape

(438557, 18)

Veri setindeki sütun isimlerini değiştirmek çok daha işlevsel olur.


In [4]:
application.rename(columns={'CODE_GENDER':'Gender','FLAG_OWN_CAR':'Car','FLAG_OWN_REALTY':'Realty',
                         'CNT_CHILDREN':'Childnmbr','AMT_INCOME_TOTAL':'TotalIncome',
                         'NAME_INCOME_TYPE':'Incometype','NAME_EDUCATION_TYPE':'Edu','NAME_FAMILY_STATUS':'Fam',
                        'NAME_HOUSING_TYPE':'Housing','DAYS_BIRTH':'Birthday', 'DAYS_EMPLOYED':'EmplymntDate',
                        'FLAG_EMAIL':'email','NAME_INCOME_TYPE':'Incometype','FLAG_WORK_PHONE':'workphn',
                         'FLAG_MOBIL':'mobil','FLAG_PHONE':'phone','CNT_FAM_MEMBERS':'famsize',
                        'OCCUPATION_TYPE':'Occupation'
                        },inplace=True)

In [5]:
application.head(3)

,ID,Gender,Car,Realty,Childnmbr,TotalIncome,Incometype,Edu,Fam,Housing,Birthday,EmplymntDate,mobil,workphn,phone,email,Occupation,famsize
0,5008804,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
1,5008805,M,Y,Y,0,427500.0,Working,Higher education,Civil marriage,Rented apartment,-12005,-4542,1,1,0,0,NaN,2.0
2,5008806,M,Y,Y,0,112500.0,Working,Secondary / secondary special,Married,House / apartment,-21474,-1134,1,0,0,0,Security staff,2.0


A check for missing values should be carried out on this dataset.

In [6]:
application.isnull().sum()

ID                   0
Gender               0
Car                  0
Realty               0
Childnmbr            0
TotalIncome          0
Incometype           0
Edu                  0
Fam                  0
Housing              0
Birthday             0
EmplymntDate         0
mobil                0
workphn              0
phone                0
email                0
Occupation      134203
famsize              0
dtype: int64

As seen above there are null values in the 'Occupation' column, so we will drop those rows entirely.

In [7]:
application=application.dropna()

In [8]:
application.info()

<class 'pandas.core.frame.DataFrame'>
Index: 304354 entries, 2 to 438556
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   ID            304354 non-null  int64  
 1   Gender        304354 non-null  object 
 2   Car           304354 non-null  object 
 3   Realty        304354 non-null  object 
 4   Childnmbr     304354 non-null  int64  
 5   TotalIncome   304354 non-null  float64
 6   Incometype    304354 non-null  object 
 7   Edu           304354 non-null  object 
 8   Fam           304354 non-null  object 
 9   Housing       304354 non-null  object 
 10  Birthday      304354 non-null  int64  
 11  EmplymntDate  304354 non-null  int64  
 12  mobil         304354 non-null  int64  
 13  workphn       304354 non-null  int64  
 14  phone         304354 non-null  int64  
 15  email         304354 non-null  int64  
 16  Occupation    304354 non-null  object 
 17  famsize       304354 non-null  float64
dtypes: float6

And a check for duplicated values.

In [9]:
application.duplicated(subset=['ID']).sum()

np.int64(23)

In the Application dataset, duplicated ID values can be removed since each ID uniquely represents a customer. However, in the Credit dataset, ID values correspond to the customer's monthly debt repayment status, so duplicates are expected and should not be removed.

In [10]:
application.drop_duplicates(subset=['ID'], inplace=True)


Make a new columns for Age and Years of Working and the "Birthday" & "EmplymntDate" columns were deleted for better understanding and for the model training.

In [11]:
application['Age']=(-application['Birthday']/365).astype(int)

For WorkingYears column, changing the negative values to years in float data type, and for the positive values, meaning 'Not Working' customers, we label them as -1 for LabelEncoding

In [12]:
application["WorkingYears"] = application["EmplymntDate"].apply(lambda x: float(-x / 365) if pd.notnull(x) and x < 0 else -1)

In [13]:
application=application.drop(['Birthday', 'EmplymntDate'], axis=1)


---

## Credit Record Dataset Feature Engineering

In [14]:
credit=pd.read_csv(r"C:\Users\selve\Desktop\credit_record.csv")
credit.head(5)

,ID,MONTHS_BALANCE,STATUS
0,5001711,0,X
1,5001711,-1,0
2,5001711,-2,0
3,5001711,-3,0
4,5001712,0,C


In [15]:
credit['STATUS'].value_counts()

STATUS
C    442031
0    383120
X    209230
1     11090
5      1693
2       868
3       320
4       223
Name: count, dtype: int64

bu veri setinde MONTHS_BALANCE ve STATUS değerlerini açıklayalım. Months_BALANCE sütunu 0'dan başlayarak geriye doğru gidiyor, -1 bir önceki ay, -2 iki ay önce vs. STATUS sütununundaki değerleri kategorik hale getirelim şuan X,C,0,1,2,3,4 değerleri var. Bu değerleri açıkla, bu değerleri , good_debt: Hiç gecikme yaşanmamış, bad_debt: Geç ödeme var, şeklinde ikiye ayıralım.

In [16]:
credit["STATUS"]=credit["STATUS"].astype(str)
credit['STATUS']=credit['STATUS'].apply(lambda x: "Good_Debt" if x in ["X", "C"] else "Bad_Debt")


In [17]:
credit['STATUS'].value_counts(normalize=True)

STATUS
Good_Debt    0.621091
Bad_Debt     0.378909
Name: proportion, dtype: float64

After changing the variables in the Status column integer to categorical values, made a new dataframe with the total counts of status by ID. 

In [18]:
status_counts = credit.groupby("ID")['STATUS'].value_counts().unstack(fill_value=0)


Adding this new dataframe to application dataframe.

In [19]:
df_app=application.set_index("ID").join(status_counts)

In [20]:
df_app.info()
print(df_app.shape)

<class 'pandas.core.frame.DataFrame'>
Index: 304331 entries, 5008806 to 6842885
Data columns (total 19 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Gender        304331 non-null  object 
 1   Car           304331 non-null  object 
 2   Realty        304331 non-null  object 
 3   Childnmbr     304331 non-null  int64  
 4   TotalIncome   304331 non-null  float64
 5   Incometype    304331 non-null  object 
 6   Edu           304331 non-null  object 
 7   Fam           304331 non-null  object 
 8   Housing       304331 non-null  object 
 9   mobil         304331 non-null  int64  
 10  workphn       304331 non-null  int64  
 11  phone         304331 non-null  int64  
 12  email         304331 non-null  int64  
 13  Occupation    304331 non-null  object 
 14  famsize       304331 non-null  float64
 15  Age           304331 non-null  int64  
 16  WorkingYears  304331 non-null  float64
 17  Bad_Debt      25134 non-null   float64
 18  Go

## **Dealing with imbalaced data**

When adding Status dataframe into Application dataframe there are going to be null values as expected. That is because Status dataframe made from Credit dataframe, and the Credit dataframe is a historical data of customers who have taken out a loan so there are less customer data. We need to balance this data.

First step is to fill those null values with zeros, then make a three classes based on the Good_debt and Bad_debt values.



In [21]:
df_app[['Good_Debt', 'Bad_Debt']] = df_app[['Good_Debt', 'Bad_Debt']].fillna(0)
((df_app['Good_Debt'] == 0) & (df_app['Bad_Debt'] == 0)).sum()

np.int64(279197)

In [22]:
df_app['debt_status'] = np.where(
    (df_app['Good_Debt'] == 0) & (df_app['Bad_Debt'] == 0), 
    'no_record',  
    np.where(
        df_app['Good_Debt'] > df_app['Bad_Debt'], 
        'good',   
        'bad'))
print(df_app['debt_status'].value_counts())

debt_status
no_record    279197
good          13097
bad           12037
Name: count, dtype: int64


### **LabelEncoding**

For model training we need to use LabelEncoder to encode categorical columns into numerical values. 

first make a copy of our dataframe. Then apply LabelEncoder into categorical columns except debt_status column.

In [23]:
df_encoded=df_app.copy()


In [24]:
from sklearn.preprocessing import LabelEncoder

categorical_cols = ['Gender', 'Car', 'Realty', 'Incometype', 'Edu', 'Fam', 'Housing', 'Occupation']

label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    df_encoded[col] = le.fit_transform(df_encoded[col].astype(str))
    label_encoders[col] = le 

In [25]:
df_encoded['debt_status'].value_counts()

debt_status
no_record    279197
good          13097
bad           12037
Name: count, dtype: int64

### **SMOTE** 



We applied SMOTE to address severe class imbalance in the dataset (e.g., far fewer bad/no_record cases than good). Imbalanced data can bias models toward the majority class, leading to poor detection of high-risk cases.

In [26]:
from imblearn.over_sampling import SMOTE

X = df_encoded.drop(['debt_status', 'Good_Debt', 'Bad_Debt'], axis=1)
y = df_encoded['debt_status']

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

print(pd.Series(y_resampled).value_counts())

debt_status
good         279197
bad          279197
no_record    279197
Name: count, dtype: int64


# **Selecting A Model**

In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_resampled, y_resampled, 
    test_size=0.3, 
    random_state=42)

print("Train set:\n", y_train.value_counts())
print("\nTest set:\n", y_test.value_counts())

Train set:
 debt_status
good         195487
no_record    195454
bad          195372
Name: count, dtype: int64

Test set:
 debt_status
bad          83825
no_record    83743
good         83710
Name: count, dtype: int64


In [ ]:
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, accuracy_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_resampled)  
y_encoded = LabelEncoder().fit_transform(y_resampled)  


models = {
    "Logistic Regression": Pipeline([
        ('scaler', StandardScaler()),
        ('model', LogisticRegression(multi_class='multinomial', max_iter=1000))
    ]),
    "KNN": Pipeline([
        ('scaler', StandardScaler()),
        ('model', KNeighborsClassifier(n_neighbors=5))
    ]),
    "Random Forest": RandomForestClassifier(random_state=42)  
}


scoring = {
    'accuracy': 'accuracy',
    'f1_weighted': 'f1_weighted'
}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


results = {}
for name, model in models.items():
    cv_results = cross_validate(model, X_resampled, y_encoded, cv=cv, scoring=scoring)
    results[name] = {
        'Accuracy (Mean)': np.mean(cv_results['test_accuracy']).round(3),
        'F1-Score (Mean)': np.mean(cv_results['test_f1_weighted']).round(3),
        'Std (Accuracy)': np.std(cv_results['test_accuracy']).round(3)
    }


results_df = pd.DataFrame(results).T
print("=== Comparison of Models (5-Fold CV) ===")
print(results_df.sort_values('F1-Score (Mean)', ascending=False))


'scaler = StandardScaler()\nX_scaled = scaler.fit_transform(X_resampled)  \ny_encoded = LabelEncoder().fit_transform(y_resampled)  \n\n\nmodels = {\n    "Logistic Regression": Pipeline([\n        (\'scaler\', StandardScaler()),\n        (\'model\', LogisticRegression(multi_class=\'multinomial\', max_iter=1000))\n    ]),\n    "KNN": Pipeline([\n        (\'scaler\', StandardScaler()),\n        (\'model\', KNeighborsClassifier(n_neighbors=5))\n    ]),\n    "Random Forest": RandomForestClassifier(random_state=42)  \n}\n\n\nscoring = {\n    \'accuracy\': \'accuracy\',\n    \'f1_weighted\': \'f1_weighted\'\n}\ncv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)\n\n\nresults = {}\nfor name, model in models.items():\n    cv_results = cross_validate(model, X_resampled, y_encoded, cv=cv, scoring=scoring)\n    results[name] = {\n        \'Accuracy (Mean)\': np.mean(cv_results[\'test_accuracy\']).round(3),\n        \'F1-Score (Mean)\': np.mean(cv_results[\'test_f1_weighted\']).round(3)

### Model Comparison Summary:

To address the imbalanced credit risk dataset, we evaluated Logistic Regression, KNN, and Random Forest using weighted F1-score and cross-validation. Random Forest outperformed others with a 0.903 F1-score and 0.903 accuracy, thanks to its ability to handle non-linear patterns, feature interactions, and class imbalance via class_weight='balanced'. While slower to train, its robust predictions and interpretability justified the choice.

**Why Random Forest?**
Unlike Logistic Regression’s linear limitations or KNN’s sensitivity to scaling, Random Forest excelled in capturing complex relationships (e.g., Income × Age) and minimizing overfitting. Its superior performance on minority classes (bad/no_record) made it the clear winner for this real-world risk prediction task.


In [34]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_resampled)  
y_encoded = LabelEncoder().fit_transform(y_resampled) 

In [35]:
# Hiperparametre optimizasyonu (Random Forest örneği)
from sklearn.model_selection import GridSearchCV
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20]
}
grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5, scoring='f1_weighted')
grid_search.fit(X_resampled, y_encoded)
print("En iyi parametreler:", grid_search.best_params_)

KeyboardInterrupt: 

In [36]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_dist = {
    'n_estimators': randint(100, 500),       # Aralık belirt
    'max_depth': randint(5, 30),             # Rastgele değer seçer
    'min_samples_split': [2, 5, 10]          # Sabit değerler de kullanılabilir
}

random_search = RandomizedSearchCV(
    RandomForestClassifier(),
    param_distributions=param_dist,
    n_iter=20,                               # Sadece 20 kombinasyon dene
    cv=3,                                    # 3-fold CV (hız için)
    scoring='f1_weighted',
    n_jobs=-1                                # Paralel çalıştır
)
random_search.fit(X_train, y_train)
print("Best parameters:", random_search.best_params_)
print("Best F1 score:", random_search.best_params_)

KeyboardInterrupt: 

In [31]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# Model ve parametreler
model = RandomForestClassifier()
params = {
    'n_estimators': randint(50, 500),
    'max_depth': randint(3, 20),
    'min_samples_split': randint(2, 20)
}

# Hızlı arama
search = RandomizedSearchCV(model, params, n_iter=10, cv=3, n_jobs=-1, scoring='f1_weighted')
search.fit(X_train, y_train)

# Sadece en iyi parametreler
print("Best Parameters:", search.best_params_)

Best Parameters: {'max_depth': 15, 'min_samples_split': 11, 'n_estimators': 341}


In [32]:
search.best_score_

np.float64(0.7861772655450645)